In [3]:
import pandas as pd
import numpy as np
from linearmodels.panel import RandomEffects, PooledOLS, PanelOLS, compare
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import chi2
from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import RandomEffects
from statsmodels.miscmodels.ordinal_model import OrderedModel
import statsmodels.api as sm

# Comprehensive Panel Data Analysis with Tests

In [4]:
file_path = 'AllData1 4.xlsx'
data = pd.read_excel(file_path, sheet_name='further cleaned')

# Clean column names
data.columns = [
    "Country",
    "Year",
    "Inflation Rate",
    "GDP per Capita",
    "General Government Debt (% of GDP)",
    "Real GDP Growth (%)",
    "Control Corruption",
    "Moody's",
    "S&P",
    "Fitch"
]

# Replace "no data" with NaN and clean formatting
data.replace("no data", pd.NA, inplace=True)
data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

# Convert numeric columns to appropriate data types
numeric_columns = [
    "Year",
    "Inflation Rate",
    "GDP per Capita",
    "General Government Debt (% of GDP)",
    "Real GDP Growth (%)",
    "Control Corruption"
]
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

data = data.dropna(subset=["Moody's", "Fitch",'S&P'], how='all')

# Set 'Country' and 'Year' as index so that bfill works per country over time
data = data.set_index(["Country", "Year"])

# Perform backward fill within each country
data = data.groupby(level="Country").bfill()

data = data.groupby(level="Country").ffill()

rows_with_missing = data[data.isna().any(axis=1)]

unique_countries = rows_with_missing.index.get_level_values("Country").unique()
data = data[~data.index.get_level_values("Country").isin(unique_countries)]

data['GDP per Capita'] = np.log(data['GDP per Capita'])

# Reset index to bring 'Country' and 'Year' back as columns
data = data.reset_index()

rating_mapping = {
    "AAA": 20, "AA+": 19, "AA": 18, "AA-": 17,
    "A+": 16, "A": 15, "A-": 14,
    "BBB+": 13, "BBB": 12, "BBB-": 11,
    "BB+": 10, "BB": 9, "BB-": 8,
    "B+": 7, "B": 6, "B-": 5,
    "CCC+": 4, "CCC": 3, "CCC-": 2,
    "CC": 1, "C": 1, "D": 1, "SD":1,
    'DDD':1, 'RD' : 1
}

# Moody's rating system mapping
moody_mapping = {
    "Aaa": 20, "Aa1": 19, "Aa2": 18, "Aa3": 17,
    "A1": 16, "A2": 15, "A3": 14,
    "Baa1": 13, "Baa2": 12, "Baa3": 11,
    "Ba1": 10, "Ba2": 9, "Ba3": 8,
    "B1": 7, "B2": 6, "B3": 5,
    "Caa1": 4, "Caa2": 3, "Caa3": 2,
    "Ca": 1, "C": 1
}

data["Moody's"] = data["Moody's"].map(moody_mapping)
data["S&P"] = data["S&P"].map(rating_mapping)
data["Fitch"] = data["Fitch"].map(rating_mapping)

# Set up panel data structure
data.set_index(["Country", "Year"], inplace=True)


C:\Users\Tiago\AppData\Local\Temp\ipykernel_7384\1441119807.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)


## Descriptive Statistics

In [5]:
descriptive_stats = data.describe()
print("Descriptive Statistics:")
print(descriptive_stats)


Descriptive Statistics:
       Inflation Rate  GDP per Capita  General Government Debt (% of GDP)  \
count     2848.000000     2848.000000                         2848.000000   
mean         6.870516        9.050959                           56.091397   
std         20.688442        1.348554                           45.837485   
min        -30.199654        5.800918                            0.100000   
25%          1.563648        8.063626                           29.900000   
50%          3.762210        9.072105                           47.100000   
75%          7.922254       10.202447                           70.325000   
max        913.213100       11.803442                          600.100000   

       Real GDP Growth (%)  Control Corruption      Moody's          S&P  \
count          2848.000000         2848.000000  2848.000000  2848.000000   
mean              3.407442            0.283091    11.849017    11.974368   
std               4.098280            1.040440     5.3

## Correlation Checks

In [6]:
correlation_matrix = data.corr()
print("Correlation Matrix:")
print(correlation_matrix)


Correlation Matrix:
                                    Inflation Rate  GDP per Capita  \
Inflation Rate                            1.000000       -0.150666   
GDP per Capita                           -0.150666        1.000000   
General Government Debt (% of GDP)        0.020621        0.093907   
Real GDP Growth (%)                      -0.069600       -0.185037   
Control Corruption                       -0.163379        0.779317   
Moody's                                  -0.207303        0.767300   
S&P                                      -0.216342        0.794682   
Fitch                                    -0.215020        0.758051   

                                    General Government Debt (% of GDP)  \
Inflation Rate                                                0.020621   
GDP per Capita                                                0.093907   
General Government Debt (% of GDP)                            1.000000   
Real GDP Growth (%)                                  

## Stationarity Tests

In [7]:
variables = ["Inflation Rate", "GDP per Capita", "General Government Debt (% of GDP)", "Real GDP Growth (%)"]
for var in variables:
    adf_result = adfuller(data[var].dropna())
    print(f"ADF Test for {var}:")
    print(f"Test Statistic: {adf_result[0]}, P-value: {adf_result[1]}")
    print("-" * 40)


ADF Test for Inflation Rate:
Test Statistic: -21.45803484888839, P-value: 0.0
----------------------------------------
ADF Test for GDP per Capita:
Test Statistic: -9.617758650411604, P-value: 1.7418270556246257e-16
----------------------------------------
ADF Test for General Government Debt (% of GDP):
Test Statistic: -8.19580401154365, P-value: 7.455324040835686e-13
----------------------------------------
ADF Test for Real GDP Growth (%):
Test Statistic: -8.023084189132776, P-value: 2.051347314838705e-12
----------------------------------------


## Multicollinearity Check

In [8]:
X = data[["Inflation Rate", "GDP per Capita", "General Government Debt (% of GDP)", "Real GDP Growth (%)",'Control Corruption']]
X["Constant"] = 1  # Add constant for VIF calculation

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print("Variance Inflation Factor (VIF):")
print(vif_data)


Variance Inflation Factor (VIF):
                             Variable         VIF
0                      Inflation Rate    1.038640
1                      GDP per Capita    2.641518
2  General Government Debt (% of GDP)    1.038188
3                 Real GDP Growth (%)    1.080727
4                  Control Corruption    2.574622
5                            Constant  118.293688


C:\Users\Tiago\AppData\Local\Temp\ipykernel_7384\3444331693.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["Constant"] = 1  # Add constant for VIF calculation


## Homoscedasticity Test

In [9]:
# Align dependent and independent variables by dropping missing values
aligned_data = pd.concat([data["Moody's"], X], axis=1)
aligned_y = aligned_data["Moody's"]
aligned_X = aligned_data.drop("Moody's", axis=1)

# Convert to NumPy arrays
aligned_y_array = aligned_y.values
aligned_X_array = aligned_X.values

# Fit the Pooled OLS model
pooled_model = PooledOLS(aligned_y, aligned_X).fit()

# Extract residuals as NumPy array
residuals = pooled_model.resids.to_numpy()

# Perform the Breusch-Pagan test
bp_test = het_breuschpagan(residuals, aligned_X_array)
print("Breusch-Pagan Test Results:")
print(f"Test Statistic: {bp_test[0]}, P-value: {bp_test[1]}")


Breusch-Pagan Test Results:
Test Statistic: 300.7142942170418, P-value: 7.03227733134298e-63


## Autocorrelation Check

In [10]:
# Durbin-Watson test for autocorrelation
dw_stat = durbin_watson(pooled_model.resids)
print("Durbin-Watson Statistic for Autocorrelation:")
print(dw_stat)


Durbin-Watson Statistic for Autocorrelation:
0.2321442723145939


## Hausman Test

In [11]:
# Fit the Fixed Effects model using PanelOLS
fe_model = PanelOLS(aligned_y, aligned_X, entity_effects=True).fit()

# Fit the Random Effects model
re_model = RandomEffects(aligned_y, aligned_X).fit()

# Hausman Test
beta_diff = fe_model.params - re_model.params
cov_diff = fe_model.cov - re_model.cov
hausman_stat = beta_diff.T @ np.linalg.inv(cov_diff) @ beta_diff
hausman_pval = chi2.sf(hausman_stat, len(beta_diff))

print("Hausman Test Results:")
print(f"Hausman Statistic: {hausman_stat}")
print(f"P-value: {hausman_pval}")

Hausman Test Results:
Hausman Statistic: 85.12705489137457
P-value: 3.1069893469565455e-16


In [12]:
# Fit the Random Effects model
re_model = RandomEffects(aligned_y, aligned_X).fit()

# Output the results
print("Random Effects Model Results:")
print(re_model.summary)

# Save results to a text file
with open("random_effects_results.txt", "w") as f:
    f.write(re_model.summary.as_text())
print("Random Effects results saved to random_effects_results.txt")

Random Effects Model Results:
                        RandomEffects Estimation Summary                        
Dep. Variable:                Moody's   R-squared:                        0.2710
Estimator:              RandomEffects   R-squared (Between):              0.7698
No. Observations:                2848   R-squared (Within):               0.2027
Date:                Sun, Dec 15 2024   R-squared (Overall):              0.6953
Time:                        20:23:06   Log-likelihood                   -5582.6
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      211.35
Entities:                         111   P-value                           0.0000
Avg Obs:                       25.658   Distribution:                  F(5,2842)
Min Obs:                       6.0000                                           
Max Obs:                       29.000   F-statistic (robust):             240.4

In [13]:
# Step 1: Align the data
aligned_data = pd.concat([data["Moody's"], X], axis=1).dropna()
if "Constant" in aligned_data.columns:
    aligned_data = aligned_data.drop("Constant", axis=1)
aligned_y = aligned_data["Moody's"].astype(int)  # Ensure dependent variable is integer-encoded
aligned_X = aligned_data.drop("Moody's", axis=1)  # Do not add a constant column here

# Step 2: Fit the Ordered Probit model
ordered_model = OrderedModel(
    aligned_y,
    aligned_X,
    distr="probit"  # Use Probit link function
)

# Fit the model
results = ordered_model.fit(method="bfgs")
print("Ordered Probit Model Results:")
print(results.summary())

# Save results to a text file
with open("ordered_probit_results.txt", "w") as f:
    f.write(results.summary().as_text())
print("Results saved to ordered_probit_results.txt")


Optimization terminated successfully.
         Current function value: 2.178421
         Iterations: 81
         Function evaluations: 84
         Gradient evaluations: 84
Ordered Probit Model Results:
                             OrderedModel Results                             
Dep. Variable:                Moody's   Log-Likelihood:                -6204.1
Model:                   OrderedModel   AIC:                         1.246e+04
Method:            Maximum Likelihood   BIC:                         1.260e+04
Date:                Sun, 15 Dec 2024                                         
Time:                        20:23:08                                         
No. Observations:                2848                                         
Df Residuals:                    2824                                         
Df Model:                           5                                         
                                         coef    std err          z      P>|z|      [0.

In [14]:
# Step 1: Predict values
predicted = results.predict().argmax(axis=1)  # Predicted categories
actual = aligned_y.values  # Actual categories

# Step 2: Calculate prediction errors
prediction_errors = predicted - actual

# Step 3: Bin errors into categories
bins = [-np.inf, -3, -2, -1, 0, 1, 2, np.inf]  # 8 edges
labels = ["<-3", "-2", "-1", "0", "1", "2", ">3"]  # 7 labels
error_bins = pd.cut(prediction_errors, bins=bins, labels=labels)  # Assign bins


# Step 4: Count occurrences for each bin
error_counts = error_bins.value_counts().reindex(labels, fill_value=0)

# Step 5: Compute percentage metrics
total_predictions = len(predicted)
percent_correct = (prediction_errors == 0).sum() / total_predictions * 100
percent_within_1 = (np.abs(prediction_errors) <= 1).sum() / total_predictions * 100
percent_within_2 = (np.abs(prediction_errors) <= 2).sum() / total_predictions * 100

# Step 6: Create the summary table
summary_table = pd.DataFrame({
    "Prediction Error (Notches)": labels,
    "Count": error_counts.values,
    "Percentage": (error_counts.values / total_predictions * 100).round(2)
})
summary_metrics = {
    "Total Observations": total_predictions,
    "% Correctly Predicted": round(percent_correct, 2),
    "% Within 1 Notch": round(percent_within_1, 2),
    "% Within 2 Notches": round(percent_within_2, 2)
}

# Display the summary table and metrics
print("Summary of Prediction Errors Table:")
print(summary_table)
print("\nSummary Metrics:")
for metric, value in summary_metrics.items():
    print(f"{metric}: {value}")

# Save results to CSV
summary_table.to_csv("summary_prediction_errors_table.csv", index=False)
with open("summary_prediction_metrics.txt", "w") as f:
    for metric, value in summary_metrics.items():
        f.write(f"{metric}: {value}\n")
print("Results saved to 'summary_prediction_errors_table.csv' and 'summary_prediction_metrics.txt'.")



Summary of Prediction Errors Table:
  Prediction Error (Notches)  Count  Percentage
0                        <-3    697       24.47
1                         -2    250        8.78
2                         -1    818       28.72
3                          0    311       10.92
4                          1    246        8.64
5                          2    173        6.07
6                         >3    353       12.39

Summary Metrics:
Total Observations: 2848
% Correctly Predicted: 10.92
% Within 1 Notch: 48.28
% Within 2 Notches: 63.13
Results saved to 'summary_prediction_errors_table.csv' and 'summary_prediction_metrics.txt'.


In [15]:
# Step 1: Align predictions and actual data
aligned_y_t = aligned_y.iloc[:-1].reset_index(drop=True)  # Ratings at time t
aligned_y_t1 = aligned_y.iloc[1:].reset_index(drop=True)  # Ratings at time t+1

predicted_t = predicted[:-1]  # Predicted ratings at time t
predicted_t1 = predicted[1:]  # Predicted ratings at time t+1

# Step 2: Determine upgrades and downgrades
upgrades_actual = (aligned_y_t1 > aligned_y_t)
downgrades_actual = (aligned_y_t1 < aligned_y_t)

upgrades_predicted = (predicted_t1 > predicted_t)
downgrades_predicted = (predicted_t1 < predicted_t)

# Step 3: Count sample upgrades and downgrades
sample_upgrades = upgrades_actual.sum()
sample_downgrades = downgrades_actual.sum()

# Step 4: Count predicted upgrades and downgrades
predicted_upgrades = upgrades_predicted.sum()
predicted_downgrades = downgrades_predicted.sum()

# Step 5: Count correct predictions
correct_upgrades_t = ((upgrades_actual & upgrades_predicted).iloc[:-1]).sum()
correct_upgrades_t1 = ((upgrades_actual & upgrades_predicted).iloc[1:]).sum()

correct_downgrades_t = ((downgrades_actual & downgrades_predicted).iloc[:-1]).sum()
correct_downgrades_t1 = ((downgrades_actual & downgrades_predicted).iloc[1:]).sum()

# Step 6: Create summary table
summary_table = pd.DataFrame({
    "Metric": [
        "Sample Upgrades", "Predicted Upgrades", "Upgrades Correctly Predicted at t", 
        "Upgrades Correctly Predicted at t+1", "Sample Downgrades", 
        "Predicted Downgrades", "Downgrades Correctly Predicted at t",
        "Downgrades Correctly Predicted at t+1"
    ],
    "Count": [
        sample_upgrades, predicted_upgrades, correct_upgrades_t, 
        correct_upgrades_t1, sample_downgrades, 
        predicted_downgrades, correct_downgrades_t, correct_downgrades_t1
    ]
})

# Display the summary table
print("Summary of Upgrades and Downgrades:")
print(summary_table)

# Save to a CSV file
summary_table.to_csv("upgrades_downgrades_summary.csv", index=False)
print("Results saved to 'upgrades_downgrades_summary.csv'.")


Summary of Upgrades and Downgrades:
                                  Metric  Count
0                        Sample Upgrades    331
1                     Predicted Upgrades    252
2      Upgrades Correctly Predicted at t     62
3    Upgrades Correctly Predicted at t+1     62
4                      Sample Downgrades    281
5                   Predicted Downgrades    237
6    Downgrades Correctly Predicted at t     81
7  Downgrades Correctly Predicted at t+1     81
Results saved to 'upgrades_downgrades_summary.csv'.


In [16]:
import pandas as pd

# Load the dataset
file_path = 'AllData1 4.xlsx' 
raw_data = pd.read_excel(file_path, sheet_name='further cleaned')

# Step 1: Filter for the specified countries and years (1998 and 2023)
selected_countries = ["Brazil", "Malaysia", "Mexico", "South Africa", "Thailand"]
filtered_data = raw_data[(raw_data['Country'].isin(selected_countries)) & (raw_data['Year'].isin([1998, 2023]))]

# Step 2: Select relevant columns for ratings
relevant_columns = ["Country", "Year", "Moody's", "S&P", "Fitch"]
ratings_data = filtered_data[relevant_columns].copy()

# Step 3: Clean and prepare data
ratings_data.replace("no data", pd.NA, inplace=True)

# Note: Instead of dropping rows with missing values, we rely on backward fill to propagate values
# If you'd rather remove rows that have no ratings in both years, uncomment the line below.
# ratings_data = ratings_data.dropna(subset=["Moody's", "S&P", "Fitch"], how='all')

# Step 4: Pivot the table to have a multi-index of columns: (RatingAgency, Year)
pivot_ratings = ratings_data.pivot(index="Country", columns="Year", values=["Moody's", "S&P", "Fitch"])

# Sort the multi-level columns by Year
# This ensures years go from left (earliest) to right (latest), which is needed for backward fill
pivot_ratings = pivot_ratings.sort_index(axis=1, level=1)

# Perform backward fill along columns to propagate the earliest available rating backwards
# axis=1 means we fill missing values from right to left within each row (country)
pivot_ratings = pivot_ratings.bfill(axis=1)

# Flatten the multi-level column index for clarity
pivot_ratings.columns = [f"{col[0]} ({col[1]})" for col in pivot_ratings.columns]

# Reset the index for display
pivot_ratings.reset_index(inplace=True)

# (Optional) Save the cleaned and formatted data
pivot_ratings.to_csv("filtered_ratings_table.csv", index=False)

# Display the resulting table
print("Ratings Table for Selected Countries (1998 and 2023):")
print(pivot_ratings)



Ratings Table for Selected Countries (1998 and 2023):
        Country Fitch (1998) Moody's (1998) S&P (1998) Fitch (2023)  \
0        Brazil           B+             B2        BB-           BB   
1      Malaysia           BB           Baa3       BBB-         BBB+   
2        Mexico           BB            Ba2         BB         BBB-   
3  South Africa           BB           Baa3        BB+          BB-   
4      Thailand          BB+            Ba1       BBB-         BBB+   

  Moody's (2023) S&P (2023)  
0            Ba2        BB-  
1             A3         A-  
2           Baa2        BBB  
3            Ba2        BB-  
4           Baa1       BBB+  


In [17]:
# -------------------------------------
# Configuration
# -------------------------------------
selected_countries = ["Brazil", "Malaysia", "Mexico", "South Africa", "Thailand"]
selected_years = [1998, 2005]

rating_cols = ["Moody's", "S&P", "Fitch"]
macro_vars = ["Inflation Rate", "GDP per Capita", 'General Government Debt (% of GDP)',
              'Real GDP Growth (%)', "Control Corruption"]

# -------------------------------------
# Filtering Data for Selected Countries and Years
# -------------------------------------
# Assuming 'data' is a DataFrame with a MultiIndex (Country, Year)
# Check that 'Country' and 'Year' are indeed the index levels.
print("Index levels:", data.index.names)

# Filter for selected countries and years
country_mask = data.index.get_level_values('Country').isin(selected_countries)
year_mask = data.index.get_level_values('Year').isin(selected_years)
filtered_data = data[country_mask & year_mask].copy()

# -------------------------------------
# Creating a Pivot Table for Ratings (Similar to the Sample Table)
# -------------------------------------
# Reset index to pivot easily
ratings_df = filtered_data[rating_cols].reset_index()

# Pivot so that we get a structure like:
# Country as rows, (Rating, Year) as columns
ratings_pivot = ratings_df.pivot(index="Country", columns="Year", values=rating_cols)

# Sort columns by year for clarity
ratings_pivot = ratings_pivot.sort_index(axis=1, level=1)

# Flatten the multi-level column names
ratings_pivot.columns = [f"{col[0]} ({col[1]})" for col in ratings_pivot.columns]

print("\nRatings Pivot Table (1998 & 2005):")
print(ratings_pivot)

# -------------------------------------
# Computing Differences between 1998 and 2005 Ratings
# -------------------------------------
# Extract columns for 1998 and 2005 for each rating
# Example: Moody's (1998) and Moody's (2005), etc.
diff_data = pd.DataFrame(index=ratings_pivot.index)
for r in rating_cols:
    col_1998 = f"{r} (1998)"
    col_2005 = f"{r} (2005)"
    if col_1998 in ratings_pivot.columns and col_2005 in ratings_pivot.columns:
        diff_data[f"{r} Change (2005-1998)"] = ratings_pivot[col_2005] - ratings_pivot[col_1998]

print("\nChange in Ratings (2005 - 1998):")
print(diff_data)

# -------------------------------------
# Running Regressions to See How Macro Variables Affect Ratings
# -------------------------------------
# We will run a regression for each country across all available years (not just 1998 and 2005)
# to understand how macro variables relate to a chosen rating (e.g., Fitch).
impact_results = []

for country in selected_countries:
    # Filter full data (all years) for this country
    c_mask = data.index.get_level_values('Country') == country
    c_data = data[c_mask].copy()
    
    # Dependent variable: Fitch rating
    y = c_data["Fitch"]
    
    # Independent variables: macro factors
    X = c_data[macro_vars]
    
    # Drop missing rows
    valid_idx = X.dropna().index.intersection(y.dropna().index)
    X = X.loc[valid_idx]
    y = y.loc[valid_idx]
    
    # Add a constant term
    X = sm.add_constant(X)
    
    # Fit a simple OLS model
    model = sm.OLS(y, X).fit()
    
    # Collect results
    coef_df = pd.DataFrame({
        "Country": country,
        "Variable": model.params.index,
        "Coefficient": model.params.values,
        "Std. Error": model.bse.values,
        "P-Value": model.pvalues.values
    })
    
    impact_results.append(coef_df)

impact_table = pd.concat(impact_results, ignore_index=True)

print("\nImpact of Variables on Fitch Ratings by Country:")
print(impact_table)

# -------------------------------------
# Formatting and Saving Results
# -------------------------------------
# You can now save your pivot tables and regression results to CSV, Excel, or format them further.
ratings_pivot.to_csv("ratings_pivot_1998_2005.csv", index=True)
diff_data.to_csv("ratings_change_1998_2005.csv", index=True)
impact_table.to_csv("fitch_model_impacts_by_country.csv", index=False)

# -------------------------------------
# Summary
# -------------------------------------
# The above code:
# 1. Filters the data to selected countries and years.
# 2. Creates a pivot table of ratings like the example.
# 3. Computes differences in ratings between 1998 and 2005.
# 4. Runs regressions to see how macro variables explain the variation in a chosen rating measure.
# Adjust column names and variable selections as needed to fully match your data and goals.


Index levels: ['Country', 'Year']

Ratings Pivot Table (1998 & 2005):
              Fitch (1998)  Moody's (1998)  S&P (1998)  Fitch (2005)  \
Country                                                                
Brazil                   7               6           8             8   
Malaysia                 9              11          11            14   
Mexico                   9               9           9            12   
South Africa             9              11          10            13   
Thailand                10              10          11            13   

              Moody's (2005)  S&P (2005)  
Country                                   
Brazil                     8           8  
Malaysia                  14          14  
Mexico                    13          12  
South Africa              13          13  
Thailand                  13          13  

Change in Ratings (2005 - 1998):
              Moody's Change (2005-1998)  S&P Change (2005-1998)  \
Country               